In [4]:
import os
import getpass

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = ""
os.environ['ACTIVELOOP_TOKEN'] = ""
embeddings = OpenAIEmbeddings()


/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [5]:
!git clone https://github.com/twitter/the-algorithm # replace any repository of your choice

Cloning into 'the-algorithm'...
remote: Enumerating objects: 9142, done.
remote: Counting objects: 100% (1216/1216), done.
remote: Compressing objects: 100% (759/759), done.
remote: Total 9142 (delta 483), reused 457 (delta 457), pack-reused 7926
Receiving objects: 100% (9142/9142), 7.52 MiB | 4.36 MiB/s, done.
Resolving deltas: 100% (3021/3021), done.


In [6]:
import os
from langchain.document_loaders import TextLoader

root_dir = './the-algorithm'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [7]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 2549, which is longer than the specified 1000
Created a chunk of size 2095, which is longer than the specified 1000
Created a chunk of size 1983, which is longer than the specified 1000
Created a chunk of size 1699, which is longer than the specified 1000
Created a chunk of size 1645, which is longer than the specified 1000
Created a chunk of size 1045, which is longer than the specified 1000
Created a chunk of size 2584, which is longer than the specified 1000
Created a chunk of size 1448, which is longer than the specified 1000
Created a chunk of size 2095, which is longer than the specified 1000
Created a chunk of size 1215, which is longer than the specified 1000
Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1685, which is longer than the specified 1000
Created a chunk of size 1354, which is longer than the specified 1000
Created a chunk of size 1340, which is longer than the specified 1000
Created a chunk of s

In [8]:
username = "gustavobarbosa060" # replace with your username from app.activeloop.ai
db = DeepLake(dataset_path=f"hub://{username}/twitter-algorithm", embedding=embeddings)
db.add_documents(texts)


Your Deep Lake dataset has been successfully created!


Creating 31314 embeddings in 63 batches of size 500:: 100%|██████████| 63/63 [53:20<00:00, 50.80s/it]

Dataset(path='hub://gustavobarbosa060/twitter-algorithm', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype        shape       dtype  compression
  -------    -------      -------     -------  ------- 
   text       text      (31314, 1)      str     None   
 metadata     json      (31314, 1)      str     None   
 embedding  embedding  (31314, 1536)  float32   None   
    id        text      (31314, 1)      str     None   


['e7570a9a-1c9b-11ef-abdd-8038fb44b484',
 'e7570c66-1c9b-11ef-abdd-8038fb44b484',
 'e7570d1a-1c9b-11ef-abdd-8038fb44b484',
 'e7570d9c-1c9b-11ef-abdd-8038fb44b484',
 'e7570e0a-1c9b-11ef-abdd-8038fb44b484',
 'e7570e78-1c9b-11ef-abdd-8038fb44b484',
 'e7570ee6-1c9b-11ef-abdd-8038fb44b484',
 'e7570fae-1c9b-11ef-abdd-8038fb44b484',
 'e757103a-1c9b-11ef-abdd-8038fb44b484',
 'e75710a8-1c9b-11ef-abdd-8038fb44b484',
 'e757110c-1c9b-11ef-abdd-8038fb44b484',
 'e7571170-1c9b-11ef-abdd-8038fb44b484',
 'e75711d4-1c9b-11ef-abdd-8038fb44b484',
 'e757122e-1c9b-11ef-abdd-8038fb44b484',
 'e7571288-1c9b-11ef-abdd-8038fb44b484',
 'e7571396-1c9b-11ef-abdd-8038fb44b484',
 'e7571418-1c9b-11ef-abdd-8038fb44b484',
 'e757147c-1c9b-11ef-abdd-8038fb44b484',
 'e75714d6-1c9b-11ef-abdd-8038fb44b484',
 'e7571530-1c9b-11ef-abdd-8038fb44b484',
 'e757158a-1c9b-11ef-abdd-8038fb44b484',
 'e75715ee-1c9b-11ef-abdd-8038fb44b484',
 'e75716ca-1c9b-11ef-abdd-8038fb44b484',
 'e757172e-1c9b-11ef-abdd-8038fb44b484',
 'e7571788-1c9b-

In [9]:
db = DeepLake(dataset_path="hub://davitbun/twitter-algorithm", read_only=True, embedding_function=embeddings)


Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://davitbun/twitter-algorithm already exists, loading from the storage


In [10]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['k'] = 10


In [11]:
def filter(x):
    if 'com.google' in x['text'].data()['value']:
        return False
    metadata = x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

# Uncomment the following line to apply custom filtering
# retriever.search_kwargs['filter'] = filter


In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model='gpt-4')
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)


/home/gustavo-barbosa/anaconda3/envs/genai/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
questions = [
    "What does favCountParams do?",
    "is it Likes + Bookmarks, or not clear from the code?",
    "What are the major negative modifiers that lower your linear ranking parameters?",   
    "How do you get assigned to SimClusters?",
    "What is needed to migrate from one SimClusters to another SimClusters?",
    "How much do I get boosted within my cluster?",   
    "How does Heavy ranker work. what are it’s main inputs?",
    "How can one influence Heavy ranker?",
    "why threads and long tweets do so well on the platform?",
    "Are thread and long tweet creators building a following that reacts to only threads?",
    "Do you need to follow different strategies to get most followers vs to get most likes and bookmarks per tweet?",
    "Content meta data and how it impacts virality (e.g. ALT in images).",
    "What are some unexpected fingerprints for spam factors?",
    "Is there any difference between company verified checkmarks and blue verified individual checkmarks?",
] 
chat_history = []

for question in questions:  
    result = qa.invoke({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")
